In [ ]:
!pip install -q yfinance

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns

# Lendo o arquivo de excel com a base de dados.
* Nesse caso está lendo a aba "cv" da coluna A até H. ✅

In [ ]:
base_dados = ('holding.xlsm')
df = pd.read_excel(base_dados, sheet_name='cv', usecols="A:H")
df

,DATA,MÊS,ANO,C/V,ATIVO,QUANTIDADE,PREÇO,VALOR TOTAL
0,2020-02-14,FEB,2020,COMPRA,BIDI4,100,16.190000,1619.000000
1,2020-02-14,FEB,2020,COMPRA,BOVA11,10,110.190000,1101.900000
2,2020-02-14,FEB,2020,COMPRA,COGN3,100,11.780000,1178.000000
3,2020-02-14,FEB,2020,COMPRA,ECOR3,100,17.120000,1712.000000
4,2020-02-17,FEB,2020,COMPRA,MRFG3,100,12.340000,1234.000000
...,...,...,...,...,...,...,...,...
1071,2023-05-31,MAY,2023,COMPRA,FLRY3,100,15.620000,1562.000000
1072,2023-06-14,JUN,2023,BONIFICAÇÃO,FLRY3,91,6.524294,593.710712
1073,2023-06-16,JUN,2023,COMPRA,EGIE3,53,45.190000,2395.070000
1074,2023-06-16,JUN,2023,COMPRA,TAEE3,100,12.620000,1262.000000


* Calculando a soma de ATIVOS na coluna QUANTIDADE ✅
* Filtrando os ativos com quantidade > 1 para remoção de alguns ativos indesejados.

In [56]:
df_total_ativos = df.groupby('ATIVO')['QUANTIDADE'].sum()
df_total_ativos = df_total_ativos[df_total_ativos >1]
df_total_ativos


ATIVO
ABCB4         900
BBAS3        1457
BBDC3         800
BBDC4         900
BBSE3        1900
DOLAR         100
EGIE3         700
FLRY3        1915
IRBR1          28
ITSA4        2500
JHSF3       10694
MXRF12        176
POUPANÇA       44
PSSA3         612
SANB3        1100
SAPR4        4478
TAEE3        1600
TRPL4        2265
Name: QUANTIDADE, dtype: int64

* Criando uma lista com os ativos indesejados e demovendo em cima com a varial df_ativos_filtrados

In [61]:
ativos_remover = ['POUPANÇA', 'MXRF12', 'IRBR1', 'DOLAR']
df_ativos_filtrados = df_total_ativos[~df_total_ativos.index.isin(ativos_remover)]
df_ativos_filtrados

14

* Convertendo a Series para DataFrame

In [62]:
df_total_acoes = df_ativos_filtrados.reset_index()
df_total_acoes

,ATIVO,QUANTIDADE
0,ABCB4,900
1,BBAS3,1457
2,BBDC3,800
3,BBDC4,900
4,BBSE3,1900
5,EGIE3,700
6,FLRY3,1915
7,ITSA4,2500
8,JHSF3,10694
9,PSSA3,612


* Obtendo a lista de Tickers

In [63]:
tickers = [ativo + '.SA' for ativo in df_total_acoes ['ATIVO']]
tickers

['ABCB4.SA',
 'BBAS3.SA',
 'BBDC3.SA',
 'BBDC4.SA',
 'BBSE3.SA',
 'EGIE3.SA',
 'FLRY3.SA',
 'ITSA4.SA',
 'JHSF3.SA',
 'PSSA3.SA',
 'SANB3.SA',
 'SAPR4.SA',
 'TAEE3.SA',
 'TRPL4.SA']

# Baixar os dados históricos de fechamento das ações

In [64]:
cotacao_atual = yf.download(tickers, period="1d")["Adj Close"]

[*********************100%***********************]  14 of 14 completed


* Ajustando o DataFrame cotacao_atual ❗️ * Tentar colocar Ações como index e nome e cotação como coluna. ❗️

In [72]:
cotacao_atual_reset = cotacao_atual.reset_index()  # Resetando o índice para transformar a data em uma coluna
cotacao_longa = cotacao_atual_reset.melt(id_vars='Date', var_name='ACOES', value_name='COTACAO')
cotacao_longa = cotacao_longa.drop(columns='Date')
cotacao_longa

,ACOES,COTACAO
0,ABCB4,17.690001
1,BBAS3,49.490002
2,BBDC3,14.540000
3,BBDC4,16.950001
4,BBSE3,30.450001
5,EGIE3,45.200001
6,FLRY3,15.650000
7,ITSA4,9.650000
8,JHSF3,4.950000
9,PSSA3,27.280001


In [73]:
cotacao_atual.columns = [col.replace('.SA', '') for col in cotacao_atual.columns]
cotacao_atual

,ABCB4,BBAS3,BBDC3,BBDC4,BBSE3,EGIE3,FLRY3,ITSA4,JHSF3,PSSA3,SANB3,SAPR4,TAEE3,TRPL4
Date,,,,,,,,,,,,,,
2023-06-16,17.690001,49.490002,14.54,16.950001,30.450001,45.200001,15.65,9.65,4.95,27.280001,14.07,4.32,12.46,24.9


# Mesclando df_total_acoes e cotacao_atual ❗️❗️❗️

In [74]:
valor_carteira = df_total_acoes.merge(cotacao_longa.T, left_on='ATIVO', right_index=True)
valor_carteira

,ATIVO,QUANTIDADE,2023-06-16 00:00:00
0,ABCB4,900,17.690001
1,BBAS3,1457,49.490002
2,BBDC3,800,14.540000
3,BBDC4,900,16.950001
4,BBSE3,1900,30.450001
5,EGIE3,700,45.200001
6,FLRY3,1915,15.650000
7,ITSA4,2500,9.650000
8,JHSF3,10694,4.950000
9,PSSA3,612,27.280001
